In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Génération d'une série temporelle synthétique (exemple) ---
np.random.seed(42)
time = np.arange(0, 2000)
series = 0.5 * time + 10 * np.sin(time / 50) + np.random.randn(2000) * 5

# --- Séparation train / validation ---
split_time = 1500
x_train = series[:split_time]
x_valid = series[split_time:]



In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

train_dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

#Pour la validation, on n'effectue pas de shuffle:
def windowed_dataset_val(series, window_size, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

val_dataset = windowed_dataset_val(x_valid, window_size, batch_size)

In [ ]:
import tensorflow as tf

def train_and_evaluate(model, train_dataset, val_dataset, epochs=20, optimizer='adam', loss='mse', metrics=['mae']):
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    # Entraînement
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=epochs
    )


    # Évaluation finale sur le dataset de validation
    results = model.evaluate(val_dataset)
    print(f"Résultats sur validation: {dict(zip(model.metrics_names, results))}")
    # Sauvegarde automatique
    model.save("trained_model.h5")
    print("Modèle sauvegardé sous le nom : trained_model.h5")
    return history


# Définition des modèles


**Modèle MLP**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_mlp_model(window_shape):
    model = models.Sequential([
        layers.Dense(30, activation="relu", input_shape=[window_shape]),
        layers.Dense(10, activation="relu"),
        layers.Dense(1)
    ])
    return model

**Modèle RNN**

In [ ]:
def create_simple_rnn_model(window_size):
    model = tf.keras.models.Sequential([
      tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 1]),
      tf.keras.layers.SimpleRNN(32),
      tf.keras.layers.Dense(1),
      tf.keras.layers.Lambda(lambda x: x * 100.0)
    ])
    return model

**Modèle LSTM**

In [ ]:
def create_lstm_model(window_size):
    model = tf.keras.models.Sequential([
      tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 1]),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(1),
      tf.keras.layers.Lambda(lambda x: x * 100.0) # Example lambda layer
    ])
    return model

**Modèle BSTM**

In [ ]:
def create_bidirectional_lstm_model(window_size):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True), input_shape=[None, 1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 100.0)
    ])
    return model

**Modèle GRU**

In [ ]:
def create_gru_model(window_size):
    model = tf.keras.models.Sequential([
        tf.keras.layers.GRU(32, return_sequences=True, input_shape=[None, 1]),
        tf.keras.layers.GRU(32),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 100.0)
    ])
    return model

In [ ]:
# Train and evaluate Simple RNN model
simple_rnn_model = create_simple_rnn_model(window_size)
print("\nTraining Simple RNN model...")
history_simple_rnn = train_and_evaluate(simple_rnn_model, train_dataset, val_dataset)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Training Simple RNN model...
Epoch 1/20
     44/Unknown 4s 10ms/step - loss: 51307.6250 - mae: 189.1169

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 50354.0117 - mae: 187.3781 - val_loss: 177825.7500 - val_mae: 415.8271
Epoch 2/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 4575.0386 - mae: 49.0446 - val_loss: 73616.8750 - val_mae: 262.3749
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 901.4067 - mae: 21.3310 - val_loss: 52684.0273 - val_mae: 219.1760
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 351.4254 - mae: 12.8041 - val_loss: 41366.8008 - val_mae: 191.4389
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 746.9627 - mae: 20.7005 - val_loss: 38782.4102 - val_mae: 184.8798
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 185.9915 - mae: 10.2799 - val_loss: 33721.1172 - val_mae: 170.6111
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 86.0310 - mae: 6.8997 - val_loss: 31104.4336 - val_mae: 163.3947
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 115.9825 - mae: 8.6074 - val_loss: 28708.0547 - val_mae: 155.4217
E

Résultats sur validation: {'loss': 23214.041015625, 'compile_metrics': 137.9710235595703}
Modèle sauvegardé sous le nom : trained_model.h5


In [ ]:
# Train and evaluate MLP model
mlp_model = create_mlp_model(window_size)
print("Training MLP model...")
history_mlp = train_and_evaluate(mlp_model, train_dataset, val_dataset)

Training MLP model...
Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 47990.8633 - mae: 183.6405 - val_loss: 2982.0864 - val_mae: 54.0870
Epoch 2/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 540.6220 - mae: 17.9326 - val_loss: 82.3684 - val_mae: 7.6084
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 42.5611 - mae: 5.2453 - val_loss: 41.0292 - val_mae: 5.0642
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 41.1321 - mae: 5.1258 - val_loss: 51.8992 - val_mae: 5.7785
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 45.4647 - mae: 5.4632 - val_loss: 85.2926 - val_mae: 7.7426
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 42.8470 - mae: 5.1781 - val_loss: 42.8803 - val_mae: 5.2226
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 38.3440 - mae: 4.9835 - val_loss: 73.5685 - val_mae: 7.1074
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 38.5376 - mae: 4.9817 - val_loss: 48.8103 - val_mae: 5.6067
Epoch 9/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss

Résultats sur validation: {'loss': 32.88377380371094, 'compile_metrics': 4.5175957679748535}
Modèle sauvegardé sous le nom : trained_model.h5


In [1]:
# Train and evaluate LSTM model
lstm_model = create_lstm_model(window_size)
print("\nTraining LSTM model...")
history_lstm = train_and_evaluate(lstm_model, train_dataset, val_dataset)

NameError: name 'create_lstm_model' is not defined

In [ ]:
# Train and evaluate Bidirectional LSTM model
bidirectional_lstm_model = create_bidirectional_lstm_model(window_size)
print("\nTraining Bidirectional LSTM model...")
history_bidirectional_lstm = train_and_evaluate(bidirectional_lstm_model, train_dataset, val_dataset)

In [ ]:
# Train and evaluate GRU model
gru_model = create_gru_model(window_size)
print("\nTraining GRU model...")
history_gru = train_and_evaluate(gru_model, train_dataset, val_dataset)